# <span style="color:#0F19C9">Contenido</span>

- [Descargar información](#descargar-información)
- [Descargar información complementaria](#descargar-información-complementaria)
- [Organizar los dataframes](#organizar-los-dataframes)
- [Eliminar valores atípicos y exportar](#eliminar-valores-atípicos-y-exportar)

# <span style="color:#0F19C9">Descargar información</span>

Vamos a utilizar la librería `InstagrAPI` para poder acceder de forma fácil a los datos que necesitamos en la plataforma Instagram. Vamos a extraer los últimos 20 post de cada uno de los artistas más importantes del género a septiembre de 2024 en Colombia que son: Los Hermanos Ariza Show, Uriel Henao, Grupo Dominio y Los 5 del Norte.

In [1]:
# Importar la librería para descargar datos
from instagrapi import Client

# Importar librerías para manipulación de dataframes
import pandas as pd
import numpy as np
import re

# Importar librería para manejo de credenciales
from dotenv import load_dotenv
import os

In [2]:
# Cargar las credenciales
load_dotenv()

# Guardar las credenciales de acceso
user = os.getenv('user_account')
password = os.getenv('user_password')

# Ingresar a Instagram con las credenciales
cl = Client()
cl.login(user, password)

True

In [3]:
# Crear listas vacías para guardar la información
artists = []
followers = []
taken_at = []
captions = []
views = []
likes = []
comments = []
type = []

# Hacer lista de artistas
accounts = ['hermanosarizashow.oficial',
            'urielhenaooficial',
            'tugrupodominio',
            'los5delnorte']

In [4]:
# Traer los post de cada artista
for artist in accounts:
    user_id = cl.user_id_from_username(artist)
    medias = cl.user_medias(user_id, 20)

    for video in range(len(medias)):
        info = medias[video]
        taken_at.append(info.taken_at)
        captions.append(info.caption_text)
        views.append(info.play_count)
        likes.append(info.like_count)
        comments.append(info.comment_count)
        video_type = info.media_type
        if video_type == 1:
            type.append('Foto')
        elif (video_type == 2) & (info.product_type == 'feed'):
            type.append('Video')
        elif (video_type == 2) & (info.product_type == 'igtv'):
            type.append('IGTV')
        elif (video_type == 2) & (info.product_type == 'clips'):
            type.append('Reel')
        elif (video_type == 8):
            type.append('Carrusel')

    artists = artists + ([artist] * len(medias))

Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/hermanosarizashow.oficial/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/hermanosarizashow.oficial/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/hermanosarizashow.oficial/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/urielhenaooficial/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/urielhenaooficial/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/urielhenaooficial/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/tugrupodominio/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/tugrupodominio/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=htt

In [5]:
# Crear dataframe de norteño a partir de la información descargada
df_norteno = pd.DataFrame({
    'Artista': artists,
    'Fecha y Hora': taken_at,
    'Tipo': type,
    'Caption': captions,
    'Likes': likes,
    'Comentarios': comments
})

In [6]:
# Modificar el nombre de los artistas
df_norteno['Artista'] = df_norteno['Artista'].map({
    'hermanosarizashow.oficial': 'Los Hermanos Ariza Show',
    'urielhenaooficial': 'Uriel Henao',
    'tugrupodominio': 'Grupo Dominio',
    'los5delnorte': 'Los 5 del Norte'
})

# Agregar la cantidad de seguidores actuales
df_norteno['Seguidores'] = df_norteno['Artista'].map({
    'Los Hermanos Ariza Show': 23100,
    'Uriel Henao': 50400,
    'Grupo Dominio': 70100,
    'Los 5 del Norte': 16000
})

In [7]:
# Mostrar 3 registros aleatorios
df_norteno.sample(3)

,Artista,Fecha y Hora,Tipo,Caption,Likes,Comentarios,Seguidores
11,Los Hermanos Ariza Show,2024-08-27 23:09:31+00:00,Video,"Todo esto es por ustedes, ¡gracias familia! ❤️...",127,6,23100
23,Uriel Henao,2024-09-05 14:04:21+00:00,Carrusel,"Un saludo para todos mis seguidores, un saludo...",611,10,50400
68,Los 5 del Norte,2024-08-31 20:00:50+00:00,Video,"📣 ¡ Ya salioooo! 🔥🎬🎥 Familia, está disponible ...",25,0,16000


# <span style="color:#0F19C9">Descargar información complementaria</span>

Utilizaremos la misma librería, pero esta vez para descargar los post de otros artistas, que, aunque no hacen parte del género directamente, nos podrían dar un panorama más general del estado actual de la red social. Los dividimos en artistas de música popular e incluimos a Pipe Bueno, Jessi Uribe, Luis Alfonso y Yeison Jiménez. Y por otro lado tenemos artistas de género norteño mundial como Los Tigres del Norte, Grupo Intocable, Los Tucanes de Tijuana y Bobby Pulido.

In [9]:
# Crear listas vacías para guardar la información
artists = []
followers = []
taken_at = []
captions = []
views = []
likes = []
comments = []
type = []

# Hacer lista de artistas
accounts_popular = ['pipebueno', 'jessiuribe3',
                    'luisalfonso', 'yeison_jimenez']
accounts_mexico = ['lostigresdelnorte', 'grupointocable',
                   'lostucanesdetijuana', 'bobbypulido425']

In [10]:
# Traer los post de cada artista
for artist in accounts_popular:
    user_id = cl.user_id_from_username(artist)
    medias = cl.user_medias(user_id, 20)

    for video in range(len(medias)):
        info = medias[video]
        taken_at.append(info.taken_at)
        captions.append(info.caption_text)
        views.append(info.play_count)
        likes.append(info.like_count)
        comments.append(info.comment_count)
        video_type = info.media_type
        if video_type == 1:
            type.append('Foto')
        elif (video_type == 2) & (info.product_type == 'feed'):
            type.append('Video')
        elif (video_type == 2) & (info.product_type == 'igtv'):
            type.append('IGTV')
        elif (video_type == 2) & (info.product_type == 'clips'):
            type.append('Reel')
        elif (video_type == 8):
            type.append('Carrusel')

    artists = artists + ([artist] * len(medias))

Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/pipebueno/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/pipebueno/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/pipebueno/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/jessiuribe3/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/jessiuribe3/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/jessiuribe3/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/luisalfonso/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/luisalfonso/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/luisalfonso/?__a=1&__d=dis) >>> 
Status 201: JSON

In [11]:
# Crear dataframe de popular a partir de la información descargada
df_popular = pd.DataFrame({
    'Artista': artists,
    'Fecha y Hora': taken_at,
    'Tipo': type,
    'Caption': captions,
    'Likes': likes,
    'Comentarios': comments
})

# Crear listas vacías para guardar la información
artists = []
followers = []
taken_at = []
captions = []
views = []
likes = []
comments = []
type = []

In [12]:
# Traer los post de cada artista
for artist in accounts_mexico:
    user_id = cl.user_id_from_username(artist)
    medias = cl.user_medias(user_id, 20)

    for video in range(len(medias)):
        info = medias[video]
        taken_at.append(info.taken_at)
        captions.append(info.caption_text)
        views.append(info.play_count)
        likes.append(info.like_count)
        comments.append(info.comment_count)
        video_type = info.media_type
        if video_type == 1:
            type.append('Foto')
        elif (video_type == 2) & (info.product_type == 'feed'):
            type.append('Video')
        elif (video_type == 2) & (info.product_type == 'igtv'):
            type.append('IGTV')
        elif (video_type == 2) & (info.product_type == 'clips'):
            type.append('Reel')
        elif (video_type == 8):
            type.append('Carrusel')

    artists = artists + ([artist] * len(medias))

Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/lostigresdelnorte/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/lostigresdelnorte/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/lostigresdelnorte/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/grupointocable/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/grupointocable/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/grupointocable/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/lostucanesdetijuana/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/lostucanesdetijuana/?__a=1&__d=dis) >>> 
Status 201: JSONDecodeError in public_request (url=https://www.instagram.com/

In [13]:
# Crear dataframe de mexico a partir de la información descargada
df_mexico = pd.DataFrame({
    'Artista': artists,
    'Fecha y Hora': taken_at,
    'Tipo': type,
    'Caption': captions,
    'Likes': likes,
    'Comentarios': comments
})

In [14]:
# Modificar el nombre de los artistas
df_popular['Artista'] = df_popular['Artista'].map({
    'pipebueno': 'Pipe Bueno',
    'jessiuribe3': 'Jessi Uribe',
    'luisalfonso': 'Luis Alfonso',
    'yeison_jimenez': 'Yeison Jiménez'
})
df_mexico['Artista'] = df_mexico['Artista'].map({
    'lostigresdelnorte': 'Los Tigres del Norte',
    'grupointocable': 'Grupo Intocable',
    'lostucanesdetijuana': 'Los Tucanes de Tijuana',
    'bobbypulido425': 'Bobby Pulido'
})

# Agregar la cantidad de seguidores actuales
df_popular['Seguidores'] = df_popular['Artista'].map({
    'Pipe Bueno': 10000000,
    'Jessi Uribe': 7000000,
    'Luis Alfonso': 2000000,
    'Yeison Jiménez': 5000000
})
df_mexico['Seguidores'] = df_mexico['Artista'].map({
    'Los Tigres del Norte': 2000000,
    'Grupo Intocable': 713000,
    'Los Tucanes de Tijuana': 1000000,
    'Bobby Pulido': 162000
})

In [15]:
# Mostrar 5 registros aleatorios del dataframe de popular
df_popular.sample(5)

,Artista,Fecha y Hora,Tipo,Caption,Likes,Comentarios,Seguidores
32,Jessi Uribe,2024-09-03 15:51:54+00:00,Video,Una cantada al lado de este monstruo y se rein...,-1,438,7000000
27,Jessi Uribe,2024-09-11 14:15:34+00:00,Carrusel,2 países en una sola madrugada #españa #aleman...,-1,99,7000000
40,Luis Alfonso,2024-09-12 23:11:36+00:00,Video,Ahora si se formo el propio descriteriooo con ...,34911,454,2000000
39,Jessi Uribe,2024-08-26 20:32:23+00:00,Video,Madrid 🇪🇸 cada vez falta menos para vernos 😍\n...,-1,58,7000000
71,Yeison Jiménez,2024-09-10 17:02:39+00:00,Carrusel,En BUCARAMANGA me siento en mi casa 🏠🫶\n\nGrac...,11349,105,5000000


In [16]:
# Mostrar 5 registros aleatorios del dataframe de México
df_mexico.sample(5)

,Artista,Fecha y Hora,Tipo,Caption,Likes,Comentarios,Seguidores
35,Grupo Intocable,2024-09-12 01:11:41+00:00,Foto,"13 SEPTIEMBRE - TAMPICO, TAM.\n15 SEPTIEMBRE -...",2858,31,713000
4,Los Tigres del Norte,2024-09-17 18:17:45+00:00,Video,¡Hoy amanecimos más felices que nunca! 🤠 Estam...,5358,89,2000000
45,Los Tucanes de Tijuana,2024-09-18 16:13:04+00:00,Foto,Los Tucanes de Tijuana @lostucanesdetijuana\n\...,765,7,1000000
74,Bobby Pulido,2024-06-18 18:03:04+00:00,Foto,¡Qué tal mi gente! Nos vemos este sábado 7 de ...,2158,60,162000
30,Grupo Intocable,2024-09-14 19:26:55+00:00,Video,"¡Tampico, Tamaulipas! 🙏🏽 ¡Qué energía y qué pú...",3606,23,713000


# <span style="color:#0F19C9">Organizar los dataframes</span>

Ahora, vamos a organizar todas las columnas en un formato que sea fácil para su estudio. También crearemos algunas métricas y les daremos nuevo orden a las columnas.

In [17]:
# Cambiar franja horaria a lugar de publicación
df_norteno['Fecha y Hora'] = df_norteno['Fecha y Hora'] \
    .dt.tz_convert('America/Bogota')
df_popular['Fecha y Hora'] = df_popular['Fecha y Hora'] \
    .dt.tz_convert('America/Bogota')
df_mexico['Fecha y Hora'] = df_mexico['Fecha y Hora'] \
    .dt.tz_convert('America/Mexico_City')

In [18]:
def count_words(text):
    '''Función para contar las palabras del caption'''
    return len(text.split())


def get_hashtags(text):
    '''Función para extraer los hashtags del caption'''
    return re.findall(r'#\w+', text)

In [19]:
# Aplicar funciones sobre los captions
df_norteno['Palabras en Caption'] = df_norteno['Caption'].apply(count_words)
df_norteno['Hashtags'] = df_norteno['Caption'].apply(get_hashtags)
df_norteno['Número de hashtags'] = df_norteno['Hashtags'].apply(len)

df_popular['Palabras en Caption'] = df_popular['Caption'].apply(count_words)
df_popular['Hashtags'] = df_popular['Caption'].apply(get_hashtags)
df_popular['Número de hashtags'] = df_popular['Hashtags'].apply(len)

df_mexico['Palabras en Caption'] = df_mexico['Caption'].apply(count_words)
df_mexico['Hashtags'] = df_mexico['Caption'].apply(get_hashtags)
df_mexico['Número de hashtags'] = df_mexico['Hashtags'].apply(len)

In [20]:
# Crear la métrica principal
df_norteno['KPI'] = df_norteno['Likes'] / df_norteno['Seguidores']
df_popular['KPI'] = df_popular['Likes'] / df_popular['Seguidores']
df_mexico['KPI'] = df_mexico['Likes'] / df_mexico['Seguidores']

In [21]:
# Organizar las columnas
df_norteno = df_norteno[['Artista', 'Seguidores', 'Tipo',
                         'Fecha y Hora', 'KPI', 'Likes',
                         'Comentarios', 'Caption',
                         'Palabras en Caption',
                         'Número de hashtags', 'Hashtags']]
df_popular = df_popular[['Artista', 'Seguidores', 'Tipo',
                         'Fecha y Hora', 'KPI', 'Likes',
                         'Comentarios', 'Caption',
                         'Palabras en Caption',
                         'Número de hashtags', 'Hashtags']]
df_mexico = df_mexico[['Artista', 'Seguidores', 'Tipo',
                       'Fecha y Hora', 'KPI', 'Likes',
                       'Comentarios', 'Caption',
                       'Palabras en Caption',
                       'Número de hashtags', 'Hashtags']]

# <span style="color:#0F19C9">Eliminar valores atípicos y exportar</span>

Finalmente, vamos a eliminar los valores atípicos basándonos en la nueva métrica que creamos para finalmente exportar nuestros tres dataframes.

In [22]:
def delete_outliers(df, column):
    '''Función para eliminar los valores atípicos de la columna "column"'''
    IQ1 = np.quantile(df[column], 0.25)
    IQ3 = np.quantile(df[column], 0.75)
    RIQ = IQ3 - IQ1
    return df[df[column].between(IQ1 - 1.5*RIQ, IQ3 + 1.5*RIQ)]

In [23]:
# Eliminar los valores atípicos de la columna KPI
df_norteno = delete_outliers(df_norteno, 'KPI')
df_popular = delete_outliers(df_popular, 'KPI')
df_mexico = delete_outliers(df_mexico, 'KPI')

In [24]:
# Exportar los dataframes a csv
df_norteno.to_csv('../Data/Norteno.csv', index=False)
df_popular.to_csv('../Data/Popular.csv', index=False)
df_mexico.to_csv('../Data/Mexico.csv', index=False)